## THIS IS USED TO SHOW THE AGENT

In [65]:
%matplotlib qt  

In [78]:
env = gym.make(
    "procgen:procgen-coinrun-v0",
    num_levels=0,
    rand_seed=42,
    #start_level=3,
    render_mode='rgb_array',  # Allows rendering with RGB output
    distribution_mode='easy',
    use_backgrounds=False
)

# Get the initial observation
obs = env.reset()

# Number of iterations you want to test
max_iteration = 1000

# Enable interactive plotting
plt.ion()
plt.show()

for i in range(max_iteration):
    # Ensure the observation is in the right shape for the agent
    action, logits, value = ppo_agent.select_action(tf.expand_dims(obs, axis=0), training=False)
    action = np.squeeze(action, axis=0)

    # Take the action in the environment
    next_obs, rew, done,  info = env.step(action)
    
    # Display the environment's rendered image (info['rgb'] for Procgen)
    plt.clf()  # Clear the current figure
    plt.imshow(info["rgb"])  # Render the RGB frame from the environment
    plt.pause(1 / 144)  # Adjust the speed of the loop if necessary
    

# Close the plotting and the environment
plt.close()
env.close()

## MAKE A FUNCTION TO PLOT REWARDS AND STEP

In [ ]:
# Pad the sequences with NaN
max_length = max(len(r) for r in episode_rewards_small)
padded_rewards = np.array([np.pad(r, (0, max_length - len(r)), constant_values=np.nan) for r in episode_rewards_small])

# Compute statistics
mean_rewards = np.nanmean(padded_rewards, axis=0)
std_rewards = np.nanstd(padded_rewards, axis=0)

# Plot
episodes = np.arange(len(mean_rewards))
plt.plot(episodes, mean_rewards, label='Mean Reward')
plt.fill_between(episodes, mean_rewards - std_rewards, mean_rewards + std_rewards, alpha=0.2, label='Std Dev')
plt.xlabel('Episodes')
plt.ylabel('Reward')
plt.title('Training Performance with Inhomogeneous Lengths')
plt.legend()
plt.grid()
plt.show()


In [ ]:

def pad_sequences(sequences):
    max_length = max(len(seq) for seq in sequences)
    padded = np.full((len(sequences), max_length), np.nan, dtype=float)
    
    for i, seq in enumerate(sequences):
        padded[i, :len(seq)] = seq
    
    return padded

# Convert episode_timesteps to float if it contains integers
episode_timesteps = [np.array(t, dtype=float) for t in episode_timesteps_small]

# Pad the sequences
padded_timesteps = pad_sequences(episode_timesteps)

# Compute statistics while ignoring NaN
mean_timesteps = np.nanmean(padded_timesteps, axis=0)
std_timesteps = np.nanstd(padded_timesteps, axis=0)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(range(len(mean_timesteps)), mean_timesteps, label='Mean Timesteps', color='blue')
plt.fill_between(range(len(mean_timesteps)), 
                 mean_timesteps - std_timesteps, 
                 mean_timesteps + std_timesteps, 
                 alpha=0.2, label='Std Dev', color='blue')
plt.xlabel('Episodes')
plt.ylabel('Timesteps')
plt.title('Episode Length Over Time')
plt.legend()
plt.grid()
plt.show()

## Plot

In [ ]:
# Define the window size
window_size = 10

# Compute sliding window sums manually
windowed_rewards = []
for rewards in episode_rewards:
    env_windowed = [sum(rewards[i:i+window_size]) for i in range(len(rewards) - window_size + 1)]
    windowed_rewards.append(env_windowed)

# Plot the sliding window cumulative rewards
plt.figure(figsize=(10, 6))
for i, rewards in enumerate(windowed_rewards):
    plt.plot(rewards, label=f"Env {i+1}")

# Add labels, title, and legend
plt.xlabel('Episodes (adjusted for window)')
plt.ylabel('Cumulative Reward (Sliding Window)')
plt.title(f'Cumulative Reward per Environment (Window Size = {window_size})')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Flatten all rewards into a single list (ignoring episode boundaries)
all_rewards_flattened = [reward for env_rewards in episode_rewards for reward in env_rewards]

# Compute cumulative mean
cumulative_rewards = np.cumsum(all_rewards_flattened) / np.arange(1, len(all_rewards_flattened) + 1)

# Plot
plt.plot(cumulative_rewards, label="Cumulative Mean Reward")
plt.xlabel("Time Steps")
plt.ylabel("Cumulative Reward")
plt.title("Cumulative Mean Reward During Training")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Flatten rewards and compute moving average
flat_rewards = [reward for env_rewards in episode_rewards for reward in env_rewards]
window_size = 10  # Smoothing window
moving_avg = np.convolve(flat_rewards, np.ones(window_size)/window_size, mode='valid')

# Plot
plt.plot(moving_avg, label="Moving Average (Window=10)")
plt.xlabel("Episodes")
plt.ylabel("Reward")
plt.title("Reward Moving Average During Training")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume `episode_rewards` is a list of lists where:
# - Outer list: Each environment
# - Inner list: Rewards per episode for that environment

num_envs = len(episode_rewards_small)  # Total number of environments

# Compute cumulative rewards per environment
cumulative_rewards_per_env = [np.cumsum(rewards) for rewards in episode_rewards_small]

# Find max episode length across all environments (to ensure fair plotting)
max_episodes = max(len(rewards) for rewards in episode_rewards_small)

# Create a figure
plt.figure(figsize=(12,6))

# Plot cumulative rewards for each environment
for i in range(num_envs):
    plt.plot(range(len(cumulative_rewards_per_env[i])), cumulative_rewards_per_env[i], 
             label=f"Env {i+1}", alpha=0.6)

# Compute and plot the average cumulative reward across all environments
avg_cumulative_rewards = np.zeros(max_episodes)

# Normalize environments with different episode lengths
for env_rewards in cumulative_rewards_per_env:
    padded_rewards = np.pad(env_rewards, (0, max_episodes - len(env_rewards)), mode='edge')  # Extend last value
    avg_cumulative_rewards += padded_rewards

avg_cumulative_rewards /= num_envs  # Get average across environments

# Plot the averaged cumulative reward trend
plt.plot(range(max_episodes), avg_cumulative_rewards, 
         label="Average Cumulative Reward", color='black', linewidth=2, linestyle="dashed")

plt.xlabel("Episode")
plt.ylabel("Cumulative Reward")
plt.title("Cumulative Reward Across Multiple Environments")
plt.legend()
plt.grid()
plt.show()


## ABLATION STUDY

### coinrun

In [5]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import tensorflow as tf
import numpy as np
import random
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt
import gc  
import os
import warnings
warnings.filterwarnings("ignore")

from utils import make_env, test_agent, load_models
from ppo import PPOAgent 

seed = 70
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

num_envs = 8
envs = gym.vector.AsyncVectorEnv([make_env(seed=seed + i,) for i in range(num_envs)])
observation_space = envs.single_observation_space._shape
action_space = envs.single_action_space.n

impala_filters = [16, 32, 32]
dense_units = 512

buffer_size = 2024  # Default buffer size
learning_rate = 1e-4
minibatch = 512 # Default minibatch
epochs = 3
gamma = 0.99
gae_lambda = 0.95 #0.9
epsilon = 0.1 #0.1  
entropy_coeff = 0.001#0.01

# PPO Agent with epsilon 0.1
ppo_agent_epsilon_0_1 = PPOAgent(
    envs=envs,
    num_actions=action_space,
    obs_shape=observation_space,
    impala_filters=impala_filters,
    dense_units=dense_units,
    num_envs=num_envs,
    buffer_size=buffer_size,
    learning_rate=learning_rate,
    minibatch=minibatch,
    epochs=epochs,
    entropy_coeff=entropy_coeff,
    model_name="250k_coinrun_andAdream_v2",
)

max_timesteps = 250000  
episode_rewards_epsilon_0_1, episode_timesteps_epsilon_0_1 = ppo_agent_epsilon_0_1.train(max_timesteps=max_timesteps)

# Write the results to a file
with open(ppo_agent_epsilon_0_1.model_name+'.txt', 'w') as f:
    f.write("Episode Rewards:\n")
    for rewards in episode_rewards_epsilon_0_1:
        f.write(f"{rewards}\n")
    
    f.write("\nEpisode Timesteps:\n")
    for timesteps in episode_timesteps_epsilon_0_1:
        f.write(f"{timesteps}\n")

print(f"Results have been written to {ppo_agent_epsilon_0_1.model_name+'.txt'}")

seeds = [12,123,62,91] 
test_envs = gym.vector.SyncVectorEnv(
    [make_env(seed=seed,num_levels=0,start_level=seed) for seed in (seeds)]
)

test_results_ppo = test_agent(agent=ppo_agent_epsilon_0_1, envs=test_envs, total_timesteps=25000, agent_type="ppo")
#test_results_random = test_agent(agent=None, envs=test_envs, total_timesteps=10000, agent_type="random")

#del ppo_agent_epsilon_0_1  # Delete the PPO agent object
#del envs  # Delete the environment object
#del test_envs

gc.collect()

Training Progress:   0%|          | 0/250000 [00:00<?, ?it/s]

tf.Tensor(0.3803576, shape=(), dtype=float32) tf.Tensor(0.1077861, shape=(), dtype=float32) tf.Tensor(0.54959357, shape=(), dtype=float32) tf.Tensor(2.2252672, shape=(), dtype=float32)
tf.Tensor(0.17356285, shape=(), dtype=float32) tf.Tensor(0.05766269, shape=(), dtype=float32) tf.Tensor(0.23625734, shape=(), dtype=float32) tf.Tensor(2.2284946, shape=(), dtype=float32)


Training Progress:   6%|▋         | 16192/250000 [05:13<1:15:30, 51.61it/s]

tf.Tensor(0.14566384, shape=(), dtype=float32) tf.Tensor(0.03285703, shape=(), dtype=float32) tf.Tensor(0.23009987, shape=(), dtype=float32) tf.Tensor(2.2431223, shape=(), dtype=float32)


Training Progress:   6%|▋         | 16192/250000 [05:30<1:15:30, 51.61it/s]

...

Training Progress:  97%|█████████▋| 242880/250000 [1:27:53<02:34, 46.06it/s]


Results have been written to 250k_coinrun_andAdream_v2.txt


Testing Agent: 100%|██████████| 25000/25000 [06:21<00:00, 65.60it/s]

PPO Agent Test Results: Avg Reward: 5.27, Std Reward: 4.99, Avg Length: 149.05, in 165 episodes


9

In [1]:
import tensorflow as tf
import numpy as np
import random
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt
import gc  
import os
import warnings
warnings.filterwarnings("ignore")

from utils import make_env, test_agent, load_models
from ppo import PPOAgent 

seed = 70
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

num_envs = 8
envs = gym.vector.AsyncVectorEnv([make_env(seed=seed + i,) for i in range(num_envs)])
observation_space = envs.single_observation_space._shape
action_space = envs.single_action_space.n

impala_filters = [16, 32, 32]
dense_units = 512

buffer_size = 128#256  # Default buffer size
learning_rate = 1e-5
minibatch = 128#128*num_envs//8 # Default minibatch
epochs = 6
gamma = 0.99
gae_lambda = 0.95 #0.9
epsilon = 0.1 #0.1  
entropy_coeff = 0.01#0.01

# PPO Agent with epsilon 0.1
ppo_agent_small = PPOAgent(
    envs=envs,
    num_actions=action_space,
    obs_shape=observation_space,
    impala_filters=impala_filters,
    dense_units=dense_units,
    num_envs=num_envs,
    buffer_size=buffer_size,
    learning_rate=learning_rate,
    minibatch=minibatch,
    epochs=epochs,
    entropy_coeff=entropy_coeff,
    model_name="250k_coinrun_andAdream_but_small_lr1",
)

max_timesteps = 200000  
episode_rewards_small, episode_timesteps_small = ppo_agent_small.train(max_timesteps=max_timesteps)

# Write the results to a file
with open(ppo_agent_small.model_name+'.txt', 'w') as f:
    f.write("Episode Rewards:\n")
    for rewards in episode_rewards_small:
        f.write(f"{rewards}\n")
    
    f.write("\nEpisode Timesteps:\n")
    for timesteps in episode_timesteps_small:
        f.write(f"{timesteps}\n")

print(f"Results have been written to {ppo_agent_small.model_name+'.txt'}")

seeds = [12,123,62,91] 
test_envs = gym.vector.SyncVectorEnv(
    [make_env(seed=seed,num_levels=0,start_level=seed) for seed in (seeds)]
)

test_results_ppo = test_agent(agent=ppo_agent_small, envs=test_envs, total_timesteps=20000, agent_type="ppo")
#test_results_random = test_agent(agent=None, envs=test_envs, total_timesteps=10000, agent_type="random")

del ppo_agent_small  # Delete the PPO agent object
del envs  # Delete the environment object
del test_envs

gc.collect()

Training Progress: 100%|█████████▉| 199680/200000 [2:13:42<00:12, 24.89it/s]  


Results have been written to 250k_coinrun_andAdream_but_small_lr1.txt


Testing Agent: 100%|██████████| 20000/20000 [04:04<00:00, 81.74it/s]


PPO Agent Test Results: Avg Reward: 5.03, Std Reward: 5.00, Avg Length: 108.20, in 183 episodes


9

In [5]:
import tensorflow as tf
import numpy as np
import random
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt
import gc  
import os
import warnings
warnings.filterwarnings("ignore")

from utils import make_env, test_agent, load_models
from ppo import PPOAgent 

seed = 70
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

num_envs = 8
envs = gym.vector.AsyncVectorEnv([make_env(seed=seed + i,) for i in range(num_envs)])
observation_space = envs.single_observation_space._shape
action_space = envs.single_action_space.n

impala_filters = [16, 32, 32]
dense_units = 512

buffer_size = 128#256  # Default buffer size
learning_rate = 1e-4
minibatch = 128#128*num_envs//8 # Default minibatch
epochs = 3
gamma = 0.99
gae_lambda = 0.95
epsilon = 0.1  
entropy_coeff = 0.01

# PPO Agent with epsilon 0.1
ppo_agent_small = PPOAgent(
    envs=envs,
    num_actions=action_space,
    obs_shape=observation_space,
    impala_filters=impala_filters,
    dense_units=dense_units,
    num_envs=num_envs,
    buffer_size=buffer_size,
    learning_rate=learning_rate,
    minibatch=minibatch,
    epochs=epochs,
    entropy_coeff=entropy_coeff,
    model_name="250k_coinrun_andAdream_but_small_3epochs",
)

max_timesteps = 200000  
episode_rewards_small, episode_timesteps_small = ppo_agent_small.train(max_timesteps=max_timesteps)

# Write the results to a file
with open(ppo_agent_small.model_name+'.txt', 'w') as f:
    f.write("Episode Rewards:\n")
    for rewards in episode_rewards_small:
        f.write(f"{rewards}\n")
    
    f.write("\nEpisode Timesteps:\n")
    for timesteps in episode_timesteps_small:
        f.write(f"{timesteps}\n")

print(f"Results have been written to {ppo_agent_small.model_name+'.txt'}")

seeds = [12,123,62,91] 
test_envs = gym.vector.SyncVectorEnv(
    [make_env(seed=seed,num_levels=0,start_level=seed) for seed in (seeds)]
)

test_results_ppo = test_agent(agent=ppo_agent_small, envs=test_envs, total_timesteps=20000, agent_type="ppo")
#test_results_random = test_agent(agent=None, envs=test_envs, total_timesteps=10000, agent_type="random")
del ppo_agent_small  # Delete the PPO agent object
del envs  # Delete the environment object
del test_envs
gc.collect()

Training Progress: 100%|█████████▉| 199680/200000 [1:23:41<00:08, 39.77it/s]


Results have been written to 250k_coinrun_andAdream_but_small_3epochs.txt


Testing Agent: 100%|██████████| 20000/20000 [04:32<00:00, 73.35it/s]


PPO Agent Test Results: Avg Reward: 5.05, Std Reward: 5.00, Avg Length: 166.24, in 111 episodes


9